In [ ]:
import pathlib
import ast
import pandas as pd

from inverse_cai.app.loader import load_json_file


def create_votes_df(results_dir: pathlib.Path) -> list[dict]:

    # load relevant data from experiment logs
    votes_per_comparison = pd.read_csv(
        results_dir / "040_votes_per_comparison.csv", index_col="index"
    )
    votes_per_comparison["votes_dicts"] = votes_per_comparison["votes"].apply(ast.literal_eval)
    # add single votes_dicts per comparison_id
    votes_per_comparison["original_index"] = votes_per_comparison.index

    # create a single list of dicts per comparison_id
    votes_per_comparison = votes_per_comparison.groupby("original_index")["votes_dicts"].apply(list).reset_index(name="votes_dicts")

    # compile list of dicts into a single dict
    votes_per_comparison["votes_dicts"] = votes_per_comparison["votes_dicts"].apply(lambda x: {k: v for d in list(x) for k, v in d.items()})

    principles_by_id: dict = load_json_file(
        results_dir / "030_distilled_principles_per_cluster.json",
    )
    comparison_df = pd.read_csv(results_dir / "000_train_data.csv", index_col="index")

    # merge original comparison data with votes per comparison
    full_df = comparison_df.merge(
        votes_per_comparison, left_index=True, right_on="original_index"
    )
    full_df["comparison_id"] = full_df.index


    new_cols = {}

    def make_vote_numeric(vote: bool | None) -> int:
        if vote is True:
            return 1
        elif vote is False:
            return -1
        elif vote is None:
            return 0
        else:
            raise ValueError(f"Invalid vote value: {vote}")

    def make_vote_string(vote: bool | None, human_pref_text: str) -> str:
        human_pref_text = human_pref_text.lower()
        human_rej_text = "text_a" if human_pref_text == "text_b" else "text_b"

        if vote is True:
            return human_pref_text
        elif vote is False:
            return human_rej_text
        elif vote is None:
            return "not_applicable"

    # add one column for each principle vote
    for principle_id in principles_by_id.keys():
        new_cols[f"vote_principle_{principle_id}"] = full_df["votes_dicts"].apply(
            lambda x: make_vote_numeric(x.get(int(principle_id), None))
        )

    for principle_id in principles_by_id.keys():
        new_cols[f"vote_principle_pref_text_{principle_id}"] = full_df[["votes_dicts", "preferred_text"]].apply(
            lambda x: make_vote_string(x["votes_dicts"].get(int(principle_id), None), x["preferred_text"]), axis=1
        )

    full_df = pd.concat([full_df, pd.DataFrame(new_cols)], axis=1)

    return full_df

In [2]:
# Add equivalent but flipped votes (flipping text order and vote), such that the vote is the same but the text order is flipped

def flip_vote_df(df: pd.DataFrame) -> pd.DataFrame:
    """Create a copy of the votes dataframe with text order and votes flipped."""
    flipped_df = df.copy()

    # Swap text A and B
    flipped_df["text_a"] = df["text_b"]
    flipped_df["text_b"] = df["text_a"]

    # Flip preferred text
    flipped_df["preferred_text"] = df["preferred_text"].map(
        {"text_a": "text_b", "text_b": "text_a"}
    )

    # Flip numeric votes (-1 becomes 1, 1 becomes -1, 0 stays 0)
    for col in df.columns:
        if col.startswith("vote_principle_") and not col.startswith("vote_principle_pref"):
            flipped_df[col] = -df[col]

    # Flip text preference votes
    for col in df.columns:
        if col.startswith("vote_principle_pref_text_"):
            flipped_df[col] = df[col].map({
                "text_a": "text_b",
                "text_b": "text_a",
                "not_applicable": "not_applicable"
            })

    return flipped_df


In [3]:
df = create_votes_df(pathlib.Path("../exp/outputs/prism_v2/results"))

In [4]:

shared_subset = df[:4000]
shared_subset.to_csv("../data/experimental/prism_v2_4k_shared_subset_w_strings.csv", index=False)

non_shared_subset = df[4000:]
non_shared_subset.to_csv("../data/experimental/prism_v2_non_shared_subset_w_strings.csv", index=False)


In [5]:
# Double the dataset by adding flipped versions
df_with_flips = pd.concat([shared_subset, flip_vote_df(shared_subset)])
df_with_flips.to_csv("../data/experimental/prism_v2_4k_w_strings_and_flipped.csv", index=False)